# Programación declarativa @ URJC
# Programación funcional
## Curso 22-23, convocatoria ordinaria (31 de octubre de 2022)
## Campus de Vicálvaro


# Definiciones auxiliares

In [1]:
import $ivy.`org.scalatest::scalatest:3.0.8`
import org.scalatest._

import $ivy.$
import org.scalatest._

### Algunas definiciones de tipos y funciones auxiliares

In [2]:
sealed abstract class Tree[A]
case class Empty[A]() extends Tree[A]
case class Node[A](left: Tree[A], root: A, right: Tree[A]) extends Tree[A]

object Tree{
    def void[A]: Tree[A] = Empty()
    def leaf[A](a: A): Node[A] = Node(Empty(), a, Empty())
    def right[A](a: A, tree: Tree[A]): Node[A] = Node(Empty(), a, tree)
    def left[A](tree: Tree[A], a: A): Node[A] = Node(tree, a, Empty())
    def node[A](left: Tree[A], a: A, right: Tree[A]): Node[A] = Node(left, a, right)
}

import Tree._

def foldTree[A, B](tree: Tree[A])(empty: B)(node: (B, A, B) => B): B = 
    tree match {
        case Empty() => empty
        case Node(left, root, right) => node(foldTree(left)(empty)(node), root, foldTree(right)(empty)(node))
    }

defined class Tree
defined class Empty
defined class Node
defined object Tree
import Tree._
defined function foldTree

In [3]:
object Signatures{
    abstract class List[A]{
        
        // Common HOFs
        def foldRight[B](nil: B)(cons: (A, B) => B): B
        def foldLeft[B](initial: B)(update: (B, A) => B): B
        def map[B](f: A => B): List[B]
        def flatMap[B](f: A => List[B]): List[B]
        def filter(f: A => Boolean): List[A]
        def forall(pred: A => Boolean): Boolean
        def exists(pred: A => Boolean): Boolean
 
        // Returns the number of elements of this list
        def length: Int
        def reverse: List[A]
    }
}

defined object Signatures

### Definiciones auxiliares sobre la correspondencia Curry-Howard y Tipos Algebraicos de Datos

In [4]:
trait Isomorphic[A, B]{
    
    def from(a: A): B
    
    def to(b: B): A
}

defined trait Isomorphic

In [5]:
type Not[P] = P => Nothing

defined type Not

# Ejercicio 1 (a)
__(2 puntos)__

__a) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar la validez del siguiente argumento lógico: 

$\{ p\vee q \vee r, p \rightarrow s, \neg r \rightarrow \neg s\} \vdash \neg r \rightarrow q$

In [ ]:
def proof[P, Q, R, S](pqr: Either[P, Either[Q, R]], 
                      ps: P => S, 
                      rs: Not[R] => Not[S]): 
                          Not[R] => Q = 
    nr => pqr match {
        case Left(p) => rs(nr)(ps(p)) : Q
        case Right(Left(q)) => q
        case Right(Right(r)) => nr(r)
    }

__b) (1 punto)__ Utiliza la correspondencia Curry-Howard para demostrar la siguiente tautología de la lógica clásica proposicional: $\vdash_c (\neg p \rightarrow p) \rightarrow p$. Para ello, utiliza como premisa adicional la ley del tercio excluso aplicada a la proposición P, es decir, demuestra la validez intuicionista del siguiente argumento lógico:

$\{ p \vee \neg p \} \vdash (\neg p \rightarrow p) \rightarrow p$

In [ ]:
def proof[P](lemp: Either[P, Not[P]]): (Not[P] => P) => P = 
    (f: Not[P] => P) => 
        lemp match {
            case Left(p) => p
            case Right(np) => f(np)
        }

# Ejercicio 1 (b)
__(2 puntos)__

__a) (1 punto)__ Utiliza la correspondencia de Curry-Howard para demostrar la validez del siguiente argumento lógico: 

$ \vdash (p \rightarrow (q \wedge r \wedge s)) \rightarrow \neg r \rightarrow \neg p$

In [2]:
def proof[P, Q, R, S]: (P => (Q, (R, S))) => Not[R] => Not[P] = 
    (f: P => (Q, (R, S))) => (nr: Not[R]) => (p: P) =>
        nr(f(p)._2._1 : R)

defined function proof

__b) (1 punto)__ Utiliza la correspondencia Curry-Howard para demostrar la siguiente tautología de la lógica clásica proposicional: $\vdash_c p \vee (p \rightarrow q)$. Para ello, utiliza como premisa adicional la ley del tercio excluso aplicada a la proposición $p$, es decir, demuestra la validez intuicionista del siguiente argumento lógico:

$\{ p \vee \neg p \} \vdash p \vee (p \rightarrow q)$

In [6]:
def proof[P, Q](lemp: Either[P, Not[P]]): Either[P, P => Q] = 
    lemp match {
        case Left(p) => Left(p)
        case Right(np) => Right(p => np(p))
    }
    

defined function proof

In [7]:
def proof[P, Q](lemp: Either[P, Not[P]]): Either[P, P => Q] = 
    lemp

defined function proof

# Ejercicio 2 (a)
__(1 punto)__

Dado el siguiente tipo de datos `Cont[A]`:

In [6]:
trait Cont[A]{
    def apply[X]: (A => X) => X
}

defined trait Cont

demuestra que, para todo tipo `A`, los tipos de datos `Cont[A]` y `A` son isomorfos:

In [18]:
class Iso[A] extends Isomorphic[Cont[A], A]{
    
    def from(c: Cont[A]): A = {
        val f: (A => A) => A = c.apply[A]
        f((a: A) => a : A) : A
    }
    
    def to(a: A): Cont[A] = 
        new Cont[A]{
            def apply[X]: (A => X) => X = 
                (f: A => X) => 
                    f(a : A) : X
        }
}

defined class Iso

In [7]:
class Iso[A] extends Isomorphic[Cont[A], A]{
    
    def from(f: Cont[A]): A = 
        f[A](identity)

    def to(a: A): Cont[A] = 
        new Cont[A]{
            def apply[X]: (A => X) => X = 
                (f: A => X) => f(a)
        }
}

defined class Iso

# Ejercicio 2 (b)
__(1 punto)__

Dado el siguiente tipo de datos `Cont[A]`:

In [ ]:
trait Cont[A]{
    def apply[X]: (Option[A] => X) => X
}

demuestra que, para todo tipo `A`, los tipos de datos `Cont[A]` y `Option[A]` son isomorfos:

In [ ]:
class Iso[A] extends Isomorphic[Cont[A], Option[A]]{
    
    def from(f: Cont[A]): Option[A] = 
        f[Option[A]](identity)

    def to(a: Option[A]): Cont[A] = 
        new Cont[A]{
            def apply[X]: (Option[A] => X) => X = 
                (f: Option[A] => X) => f(a)
        }
}

# Ejercicio 3
__(4 puntos)__

Considérense las dos funciones siguientes, `parse` y `validate`:


In [ ]:
def parse(l: List[String]): Option[List[Int]] = 
    l match {
        case Nil => Some(Nil)
        case head :: tail => 
            val tailParse: Option[List[Int]] = parse(tail)
            val headInt: Option[Int] = head.toIntOption
            (headInt, tailParse) match {
                case (Some(h), Some(t)) => Some(h :: t)
                case _ => None
            }
    }

In [ ]:
def validate(l: List[Int]): Option[List[Int]] = 
    l match {
        case Nil => Some(Nil)
        case head :: tail => 
            val tailValidate: Option[List[Int]] = validate(tail)
            val headInt: Option[Int] = if (head < 0) None else Some(head)
            (headInt, tailValidate) match {
                case (Some(h), Some(t)) => Some(h :: t)
                case _ => None
            }
    }

In [ ]:
def ints(l: List[Double]): Option[List[Int]] = 
    l match {
        case Nil => Some(Nil)
        case head :: tail => 
            val tailInts: Option[List[Int]] = toInt(tail)
            val headInt: Option[Int] = if (!head.isValidInt) None else Some(head.toInt)
            (headInt, tailInts) match {
                case (Some(h), Some(t)) => Some(h :: t)
                case _ => None
            }
    }

La función `parse` traduce todas las cadenas de caracteres de una lista a sus representaciones numéricas correspondientes. Si alguna de las cadenas no representa un número, entonces la función devuelve `None`. Por ejemplo: 

In [ ]:
parse(List("1", "2", "3", "4")) == Some(List(1, 2, 3, 4))
parse(List("1", "2", "aaa", "4")) == None

La función `validate` comprueba que todos los números de una lista son mayores o iguales a cero. Si se cumple esta condición, entonces la función devuelve la misma lista de entrada; en otro caso, devuelve `None`.

In [ ]:
validate(List(0,1,2,3,4)) == Some(List(0, 1, 2, 3, 4))
validate(List(0,1,-2)) == None

La función `ints` comprueba que todos los números de tipo `Double` (equivalentes al tipo `double` de Java) son números enteros, es decir, que su parte fraccional es zero (ese es el cometido de la función `_.isValidInt`). Si se cumple esta condición, entonces la función devuelve la conversión a enteros de los números de tipo `Double` (realizada mediante la función `_.toInt`); en otro caso, devuelve `None`. Por ejemplo: 

In [ ]:
ints(List(1.0, 2.0, 3.0, 4.0, 5.0)) == Some(List(1, 2, 3, 4, 5))
ints(List(1.0, 2.2, 3.333, 4.0)) == None

El procesamiento en todos estos casos es similar: tenemos una función que se aplica a cada elemento de la lista, y que podría resultar exitosa o no (el éxito o fracaso se indica mediante un valor de tipo `Option`). Si esta función se puede aplicar con éxito a _todos_ los elementos, se devuelve una lista con todos los resultados; en otro caso, se devuelve `None`.

__a) (2 puntos)__ Implementa una función de orden superior que encapsule el patrón de diseño descrito anteriormente utilizado en las funciones `parse` y `validate`. A esta función la denominaremos `traverse`:

In [ ]:
def traverse[A, B](l: List[A])(f: A => Option[B]): Option[List[B]] = 
    l match {
        case Nil => Some(Nil)
        case head :: tail => 
            val tailTraverse: Option[List[B]] = traverse(tail)(f)
            val headB: Option[B] = f(head)
            (headB, tailTraverse) match {
                case (Some(h), Some(t)) => Some(h :: t)
                case _ => None
            }
    }

__b) (2 puntos)__ Reimplementa las funciones `parse` y `validate` de una manera más modular reutilizando la función de orden superior `traverse` implementada en el apartado anterior. 

In [ ]:
def parse(l: List[String]): Option[List[Int]] = 
    traverse(l)(_.toIntOption)

In [ ]:
def validate(l: List[Int]): Option[List[Int]] = 
    traverse(l)(head => if (head < 0) None else Some(head))

In [ ]:
def ints(l: List[Double]): Option[List[Int]] = 
    traverse(l)(head => if (!head.isValidInt) None else Some(head.toInt))

# Ejercicio 4
__(3 puntos)__

Dado un árbol binario se desea implementar una función `breadth` que devuelva los nodos pertenecientes a un nivel determinado del árbol. Por ejemplo:

In [9]:
class TestBreadth(breadth: Tree[Int] => Int => List[Int]) extends FlatSpec with Matchers{
    "breadth path" should "work" in {
        List(0,1) map breadth(void) shouldBe 
            List(Nil, Nil)
        
        List(0,1,2,3,4,5) map breadth(left(left(right(3,right(2,leaf(1))), 4), 5)) shouldBe 
            List(List(5), List(4), List(3), List(2), List(1), Nil)
        
        List(0,1,2,3,4,5) map breadth(node(left(leaf(4), 1), 0, node(leaf(3), 2, right(2, right(4, leaf(5)))))) shouldBe 
            List(List(0), List(1, 2), List(4, 3, 2), List(4), List(5), Nil)
        
        breadth(node(left(leaf(4), 1), 0, node(leaf(3), 2, right(2, right(4, leaf(5))))))(2) shouldBe 
            List(4) ++ List(3,2) // List(4,3,2)
        breadth(left(leaf(4), 1))(1) shouldBe List(4)
        breadth(node(leaf(3), 2, right(2, right(4, leaf(5)))))(1) shouldBe List(3,2)
    }
}

defined class TestBreadth

In [4]:
def breadth[A](t: Tree[A], l: Int): List[A] = 
    ??? 

defined function breadth

In [4]:
run(new TestBreadth(breadth[Int] : Tree[Int] => Int => List[Int]))

cell5.sc:1: type mismatch;
 found   : (ammonite.$sess.cell4.wrapper.cell2.Tree[Int], Int) => List[Int]
 required: cell5.this.cell2.Tree[Int] => (Int => List[Int])
val res5 = run(new TestBreadth(breadth[Int] : Tree[Int] => Int => List[Int]))
                                      ^
Compilation Failed

In [5]:
run(new TestBreadth((t : Tree[Int]) => (i: Int) => breadth(t, i) : List[Int]))

cell3$Helper$TestBreadth:
breadth path
- should work *** FAILED ***
  scala.NotImplementedError: an implementation is missing
  at scala.Predef$.$qmark$qmark$qmark(Predef.scala:344)
  at ammonite.$sess.cell4$Helper.breadth(cell4.sc:2)
  at ammonite.$sess.cell5$Helper.$anonfun$res5$2(cell5.sc:1)
  at ammonite.$sess.cell5$Helper.$anonfun$res5$2$adapted(cell5.sc:1)
  at scala.collection.immutable.List.map(List.scala:246)
  at ammonite.$sess.cell3$Helper$TestBreadth.$anonfun$new$1(cell3.sc:3)
  at org.scalatest.OutcomeOf.outcomeOf(OutcomeOf.scala:85)
  at org.scalatest.OutcomeOf.outcomeOf$(OutcomeOf.scala:83)
  at org.scalatest.OutcomeOf$.outcomeOf(OutcomeOf.scala:104)
  at org.scalatest.Transformer.apply(Transformer.scala:22)
  ...


In [6]:
run(new TestBreadth((breadth[Int]  _).curried))

cell7.sc:1: missing argument list for method breadth in class Helper
Unapplied methods are only converted to functions when a function type is expected.
You can make this conversion explicit by writing `breadth _` or `breadth(_,_)` instead of `breadth`.
val res7 = run(new TestBreadth(breadth[Int].curried))
                                      ^
Compilation Failed

In [13]:
def breadth[A](t: Tree[A])(l: Int): List[A] = 
    t match {
        case Empty() => Nil : List[A]
        case Node(left, root, right) => 
            if (l == 0) List(root)
            else {
                val leftSol: List[A] = breadth(left)(l-1)
                val rightSol: List[A] = breadth(right)(l-1)
                leftSol ++ rightSol : List[A]
            }
    }
            

defined function breadth

In [17]:
def breadth[A](t: Tree[A]): Int => List[A] = 
    t match {
        case Empty() => 
            _ => Nil : List[A]
        case Node(left, root, right) => 
            val leftSol: Int => List[A] = breadth(left)
            val rightSol: Int => List[A] = breadth(right)
            l => leftSol(l-1) ++ rightSol(l-1) : List[A]
    }
            

defined function breadth

In [15]:
def breadth[A](t: Tree[A])(l: Int): List[A] = 
    t match {
        case Empty() => Nil : List[A]
        case Node(_, root, _) if l == 0 => List(root)
        case Node(left, _, right) => breadth(left)(l-1) ++ breadth(right)(l-1)
    }
            

defined function breadth

In [16]:
run(new TestBreadth(breadth))

cell9$Helper$TestBreadth:
breadth path
- should work


__a) (1,5 puntos)__ Implementa la función `breadth` recursivamente.

In [14]:
def breadth[A](tree: Tree[A]): Int => List[A] = 
    level => tree match {
        case Empty() => List()
        case Node(_, root, _) if level == 0 => List(root)
        case Node(left, _, right) => 
            breadth(left)(level-1) ++ breadth(right)(level-1)
    }

defined function breadth

In [15]:
def breadth[A](tree: Tree[A])(level: Int): List[A] = 
    tree match {
        case Empty() => List()
        case Node(_, root, _) if level == 0 => List(root)
        case Node(left, _, right) => 
            breadth(left)(level-1) ++ breadth(right)(level-1)
    }

defined function breadth

In [25]:
def breadth[A](tree: Tree[A]): Int => List[A] = {
    def aux(tree: Tree[A], level: Int, acc: List[A]): List[A] = 
        tree match {
            case Empty() => acc
            case Node(_, root, _) if level == 0 => root :: acc
            case Node(left, _, right) => 
                aux(left, level-1, aux(right, level-1, acc))
        }
    
    level => aux(tree, level, List())
}

defined function breadth

In [16]:
run(new TestBreadth(breadth))

cmd12$Helper$TestBreadth:
breadth path
- should work


__b) (1,5 puntos)__ Implementa la función `breadth` mediante la función de orden superior `foldTree`.

In [17]:
def breadth[A](tree: Tree[A]): Int => List[A] = 
    foldTree(tree)((_: Int) => List[A]())(
        (breadthL, root, breadthR) => 
            level => if (level == 0) List(root)
                     else breadthL(level-1) ++ breadthR(level-1)
    )

defined function breadth

In [28]:
run(new TestBreadth(breadth))

cmd12$Helper$TestBreadth:
breadth path
- should work
